In [12]:
import kagglehub
from pathlib import Path
import shutil, random

root = Path(kagglehub.dataset_download("imsparsh/flowers-dataset"))

src = (root / "train") if (root / "train").is_dir() else root

classes = [d.name for d in src.iterdir()
           if d.is_dir() and d.name not in ("train","test")]

out = Path("flower_dataset")
for split in ("train","val"):
    for cls in classes:
        (out / split / cls).mkdir(parents=True, exist_ok=True)

random.seed(42)
for cls in classes:
    imgs = list((src/cls).glob("*.*"))
    random.shuffle(imgs)
    cut = int(0.7 * len(imgs))
    for img in imgs[:cut]:
        shutil.copy2(img, out/"train"/cls/img.name)
    for img in imgs[cut:]:
        shutil.copy2(img, out/"val"/cls/img.name)

for split in ("train","val"):
    print(split, {cls: len(list((out/split/cls).iterdir())) for cls in classes})


train {'dandelion': 452, 'daisy': 350, 'sunflower': 346, 'tulip': 424, 'rose': 347}
val {'dandelion': 194, 'daisy': 151, 'sunflower': 149, 'tulip': 183, 'rose': 150}


In [18]:
import os
import shutil
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau

# -------- ECA Attention Module --------
class ECALayer(nn.Module):
    def __init__(self, channel, k_size=None):
        super().__init__()
        if k_size is None:
            t = int(abs((torch.log2(torch.tensor(channel, dtype=torch.float32)) / 2 + 1)))
            k_size = t if t % 2 else t + 1
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.conv = nn.Conv1d(1, 1, kernel_size=k_size, padding=(k_size-1)//2, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        y = self.avg_pool(x)
        y = y.view(x.size(0), 1, x.size(1))
        y = self.conv(y)
        y = self.sigmoid(y).view(x.size(0), x.size(1), 1, 1)
        return x * y.expand_as(x)

# -------- Opti-SA Model Definition --------
class OptiSA(nn.Module):
    def __init__(self, num_classes=5):
        super().__init__()
        backbone = models.shufflenet_v2_x1_0(pretrained=True)
        self.features = nn.Sequential(
            backbone.conv1,
            backbone.maxpool,
            backbone.stage2,
            backbone.stage3,
            backbone.stage4,
            backbone.conv5,
        )
        in_feats = backbone.fc.in_features
        self.eca = ECALayer(channel=in_feats)
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.dropout = nn.Dropout(0.25)
        self.fc = nn.Linear(in_feats, num_classes)

    def forward(self, x):
        x = self.features(x)
        x = self.eca(x)
        x = self.pool(x).flatten(1)
        x = self.dropout(x)
        return self.fc(x)


def train_and_validate(data_dir='flower_dataset', batch_size=45, epochs=5, lr=1e-4):
    for split in ['train', 'val']:
        for junk in ['train', 'test']:
            path = os.path.join(data_dir, split, junk)
            if os.path.isdir(path):
                shutil.rmtree(path)

    train_tf = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.RandomRotation(90),
        transforms.ColorJitter(0.3, 0.3),
        transforms.RandomApply([transforms.GaussianBlur(3)], p=0.3),
        transforms.ToTensor(),
        transforms.Normalize([0.5] * 3, [0.5] * 3),
    ])
    val_tf = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.5] * 3, [0.5] * 3),
    ])

    train_path = os.path.join(data_dir, 'train')
    val_path = os.path.join(data_dir, 'val')
    train_ds = datasets.ImageFolder(train_path, transform=train_tf)
    val_ds = datasets.ImageFolder(val_path, transform=val_tf)
    print(f"Detected classes: {train_ds.classes}")

    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
    val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = OptiSA(num_classes=len(train_ds.classes)).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
    scheduler = ReduceLROnPlateau(optimizer, 'min', factor=0.1, patience=5, verbose=True)

    best_acc = 0.0
    for epoch in range(1, epochs + 1):
        # Training
        model.train()
        total_loss = correct = total = 0
        for imgs, labels in train_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item() * imgs.size(0)
            correct += (outputs.argmax(1) == labels).sum().item()
            total += imgs.size(0)
        train_loss = total_loss / total
        train_acc = correct / total

        # Validation
        model.eval()
        total_loss = correct = total = 0
        with torch.no_grad():
            for imgs, labels in val_loader:
                imgs, labels = imgs.to(device), labels.to(device)
                outputs = model(imgs)
                loss = criterion(outputs, labels)
                total_loss += loss.item() * imgs.size(0)
                correct += (outputs.argmax(1) == labels).sum().item()
                total += imgs.size(0)
        val_loss = total_loss / total
        val_acc = correct / total

        scheduler.step(val_loss)
        print(f"Epoch {epoch}/{epochs} | "
              f"Train loss={train_loss:.4f}, acc={train_acc:.4f} | "
              f"Val loss={val_loss:.4f}, acc={val_acc:.4f}")

        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), 'best_opti_sa.pth')

    print(f"Training complete. Best validation accuracy: {best_acc:.4f}")



train_and_validate(data_dir='flower_dataset')

Detected classes: ['daisy', 'dandelion', 'rose', 'sunflower', 'tulip']
Epoch 1/5 | Train loss=1.5968, acc=0.3538 | Val loss=1.5716, acc=0.6227
Epoch 2/5 | Train loss=1.5344, acc=0.6660 | Val loss=1.4025, acc=0.7473
Epoch 3/5 | Train loss=1.3863, acc=0.7551 | Val loss=1.2055, acc=0.8162
Epoch 4/5 | Train loss=1.2159, acc=0.8103 | Val loss=1.0417, acc=0.8634
Epoch 5/5 | Train loss=1.0572, acc=0.8213 | Val loss=0.8852, acc=0.8863
Training complete. Best validation accuracy: 0.8863


In [20]:
import os
import shutil
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau

# -------- ECA Attention Module --------
class ECALayer(nn.Module):
    def __init__(self, channel, k_size=None):
        super().__init__()
        if k_size is None:
            t = int(abs((torch.log2(torch.tensor(channel, dtype=torch.float32)) / 2 + 1)))
            k_size = t if t % 2 else t + 1
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.conv = nn.Conv1d(1, 1, kernel_size=k_size, padding=(k_size-1)//2, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        y = self.avg_pool(x)
        y = y.view(x.size(0), 1, x.size(1))
        y = self.conv(y)
        y = self.sigmoid(y).view(x.size(0), x.size(1), 1, 1)
        return x * y.expand_as(x)

class OptiSA(nn.Module):
    def __init__(self, num_classes=5):
        super().__init__()
        backbone = models.shufflenet_v2_x1_0(pretrained=True)
        self.features = nn.Sequential(
            backbone.conv1,
            backbone.maxpool,
            backbone.stage2,
            backbone.stage3,
            backbone.stage4,
            backbone.conv5,
        )
        in_feats = backbone.fc.in_features
        self.eca = ECALayer(channel=in_feats)
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.dropout = nn.Dropout(0.25)
        self.fc = nn.Linear(in_feats, num_classes)

    def forward(self, x):
        x = self.features(x)
        x = self.eca(x)
        x = self.pool(x).flatten(1)
        x = self.dropout(x)
        return self.fc(x)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = OptiSA(num_classes=5).to(device)
model.load_state_dict(torch.load('best_opti_sa.pth', map_location=device))
model.eval()

val_tf = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3),
])

img_path = '/content/sun.jpeg'
img = Image.open(img_path).convert('RGB')

x = val_tf(img).unsqueeze(0).to(device)

with torch.no_grad():
    logits = model(x)
    probs  = torch.softmax(logits, dim=1)
    pred_idx = probs.argmax(dim=1).item()

classes = ['daisy','dandelion','rose','sunflower','tulip']
print(f"Predicted class: {classes[pred_idx]}  (p={probs[0,pred_idx]:.4f})")


Predicted class: sunflower  (p=0.3386)
